In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [5]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/ML/分类/")
!ls

dataset  image	    model1.h5  model3.h5    test1.ipynb        test-vgg.ipynb
drive	 keras-vis  model2.h5  resize_data  test_Resnet.ipynb  test-WRN.ipynb


In [0]:
!pip uninstall tensorflow
!pip uninstall keras
!pip uninstall keras-vis

In [0]:
!pip install keras
!pip install tensorflow==1.12
!pip install git+https://github.com/raghakot/keras-vis.git

In [0]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
images=[]
labels=[]
def read_image(imageName):
    im = Image.open(imageName).convert('RGB')
    data = np.array(im)
    return data

In [0]:
# 读取在words里面有几个文件夹
text = os.listdir('./resize_data')
# 把文件夹里面的图片和其对应的文件夹的名字也就是对应的字
for textPath in text:
    for fn in os.listdir(os.path.join('resize_data', textPath)):
        if fn.endswith('.jpg'):
            fd = os.path.join('./resize_data', textPath, fn)
            images.append(read_image(fd))
            labels.append(textPath)
            


In [0]:
X = np.array(images)
y = np.array(list(map(int, labels))) 

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=30)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, 8)
Y_test = np_utils.to_categorical(y_test, 8)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Using TensorFlow backend.


(3200, 224, 224, 3)
(3200,)
(800, 224, 224, 3)
(800,)


In [0]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
from keras.optimizers import SGD

base_model = ResNet50(weights = 'imagenet',include_top = False,pooling = 'avg',
                      input_shape=(224, 224, 3))




In [0]:
predictions = Dense(2048, activation='relu')(base_model.output)
predictions = Dropout(0.5)(predictions)
predictions = Dense(8, activation='softmax')(predictions)



model = Model(inputs=base_model.input, outputs=predictions, name='Resnet50')
sgd = SGD(lr=0.01, decay=1e-5)

In [0]:
model.summary()

In [0]:
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [0]:
print('开始训练')
history=model.fit(X_train, y_train, batch_size=50, epochs=100,validation_data=(X_test, y_test))

In [0]:
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# model.save('model2.h5')
from keras.models import load_model
model=load_model('model2.h5')

In [0]:
model.summary()

In [0]:
from keras.applications import ResNet50
from vis.utils import utils
from keras import activations

# Hide warnings on Jupyter Notebook
import warnings
warnings.filterwarnings('ignore')

layer_idx = utils.find_layer_idx(model, 'dense_6')

model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

In [0]:
from vis.utils import utils
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6)

img1 = utils.load_img('../ML/image/1.jpg', target_size=(224, 224))
f, ax = plt.subplots(1, 1)
ax.imshow(img1)

In [0]:
from vis.visualization import visualize_saliency, overlay
from vis.utils import utils
from keras import activations

# Utility to search for layer index by name. 
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(model, 'dense_6')

f, ax = plt.subplots(1, 1)  

grads = visualize_saliency(model, layer_idx, filter_indices=0, seed_input=img1)
    
# visualize grads as heatmap
ax.imshow(grads, cmap='jet')

In [0]:
for modifier in ['guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 1)
    plt.suptitle(modifier)
       
    
    grads = visualize_saliency(model, layer_idx, filter_indices=0, 
                                   seed_input=img1, backprop_modifier=modifier)
    # Lets overlay the heatmap onto original image.    
    ax.imshow(grads, cmap='jet')

In [0]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam

penultimate_layer = utils.find_layer_idx(model, 'res5c_branch2c')

for modifier in [None, 'guided', 'relu']:
    plt.figure()
    f, ax = plt.subplots(1, 1)
    plt.suptitle("vanilla" if modifier is None else modifier)
      
    # 20 is the imagenet index corresponding to `ouzel`
    grads = visualize_cam(model, layer_idx, filter_indices=0, 
                              seed_input=img1, penultimate_layer_idx=penultimate_layer,
                              backprop_modifier=modifier)        
    # Lets overlay the heatmap onto original image.    
    jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
    ax.imshow(overlay(jet_heatmap, img1))

In [0]:
import tensorflow as tf
import numpy as np
 
def py_func(func, inp, Tout, stateful = True, name=None, grad_func=None):  
    rand_name = 'PyFuncGrad' + str(np.random.randint(0,1E+8))  
    tf.RegisterGradient(rand_name)(grad_func)  
    g = tf.get_default_graph()  
    with g.gradient_override_map({'PyFunc':rand_name}):  
        return tf.py_func(func,inp,Tout,stateful=stateful, name=name)  
  
def coco_forward(xw, y, m, name=None):  
    #pdb.set_trace()  
    xw_copy = xw.copy()  
    num = len(y)  
    orig_ind = range(num)  
    xw_copy[orig_ind,y] -= m  
    return xw_copy  
  
def coco_help(grad,y):  
    grad_copy = grad.copy()  
    return grad_copy  
  
def coco_backward(op, grad):  
      
    y = op.inputs[1]  
    m = op.inputs[2]  
    grad_copy = tf.py_func(coco_help,[grad,y],tf.float32)  
    return grad_copy,y,m  
  
def coco_func(xw,y,m, name=None):  
    with tf.op_scope([xw,y,m],name,"Coco_func") as name:  
        coco_out = py_func(coco_forward,[xw,y,m],tf.float32,name=name,grad_func=coco_backward)  
        return coco_out  
  
def cos_loss(x, y,  num_cls, reuse=False, alpha=0.25, scale=64,name = 'cos_loss'):  
    ''''' 
    x: B x D - features 
    y: B x 1 - labels 
    num_cls: 1 - total class number 
    alpah: 1 - margin 
    scale: 1 - scaling paramter 
    '''  
    # define the classifier weights  
    xs = x.get_shape()  
    y = tf.reshape(tf.cast(y, dtype = tf.int32),[-1])
    with tf.variable_scope('centers_var',reuse=reuse) as center_scope:  
        w = tf.get_variable("centers", [xs[1], num_cls], dtype=tf.float32,   
            initializer=tf.contrib.layers.xavier_initializer(),trainable=True)  
     
    #normalize the feature and weight  
    #(N,D)  
    x_feat_norm = tf.nn.l2_normalize(x,1,1e-10)  
    #(D,C)  
    w_feat_norm = tf.nn.l2_normalize(w,0,1e-10)  
      
    # get the scores after normalization   
    #(N,C)  
    xw_norm = tf.matmul(x_feat_norm, w_feat_norm)    
    #value = tf.identity(xw)  
    #substract the marigin and scale it  
    value = coco_func(xw_norm,y,alpha) * scale  
      
    # compute the loss as softmax loss  
    cos_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=value))  
  
    return cos_loss